<a href="https://colab.research.google.com/github/thiagorjes/DeepVGL/blob/main/DeepVGL_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Execute os passos descritos em:**
https://github.com/LCAD-UFES/carmen_lcad/blob/master/src/deep_vgl/treino.md

# Usando os arquivos gerados no processo descrito no link acima, você pode treinar a DARKNET nas etapas a seguir.

# A implementação da Darknet19 é parte da framework **lightnet**

# **1 - Instalar as dependências**

In [1]:
!pip3 install torch
!pip3 install torchvision
!pip3 install GPUtil
!pip3 install lightnet
!pip3 install scikit-image


  Using cached https://files.pythonhosted.org/packages/dd/b9/824df420f6abf551e41bbaacbaa0be8321dc104f9f3803051513844dc310/torch-1.8.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/7a/e881b5abb54db0e6e671ab088d079c57ce54e8a01a3ca443f561ccadb37e/typing_extensions-3.7.4.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/74/a0/4bfa036c5b88444e95d0b3b24c6e0d6047b9c5920572c4be3135e24f15ba/torchvision-0.9.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dd/b9/824df420f6abf551e41bbaacbaa0be8321dc104f9f3803051513844dc310/torch-1.8.1-cp36-cp36m-manylinux1_x86_64.whl
  Usi

  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl


# **2 - Definição da rede**

In [6]:
import lightnet as ln
import torch
import torchvision
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import brambox as bb
import torch.nn as nn
import torch.nn.functional as F
# Settings
ln.logger.setConsoleLevel('ERROR')             # Only show error log messages
bb.logger.setConsoleLevel('ERROR')    

import lightnet as ln
import torch

__all__ = ['params']

def load_classes(labels_list):
    labels=[]
    with open(labels_list) as file:
        labels = [line.strip().replace('B','').replace('E','') for line in file]
    return labels


params = ln.engine.HyperParameters( 
    # Network
    class_label_map = load_classes('../labels_3_1.txt'),
    _input_dimension = (448, 448),
    _batch_size = 128,
    _mini_batch_size = 1,
    _max_batches = 40545,

    # Loss
    _coord_scale = 1.0,
    _object_scale = 5.0,
    _noobject_scale = 1.0,
    _class_scale = 1.0,

    # Dataset
    _train_set = '/dados/ufes/train.list',
    _val_set = '/dados/ufes/valid.list',
    _filter_anno = 'ignore',

    # Data Augmentation
    _jitter = .3,
    # _flip = .5,
    _angle = 7,
    _hue = .1,
    _saturation = .75,
    # _value = .75,
)

# Network
def init_weights(m):
    if isinstance(m, torch.nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight, nonlinearity='leaky_relu')

def new_network(class_label_map):
    params.network = ln.models.Darknet19(len(class_label_map))
    params.network.apply(init_weights)
    params.loss = nn.CrossEntropyLoss()
    

# **3 - Definir o dataset loader**

In [7]:
import copy
import logging
from PIL import Image
import torch
from torchvision import transforms as tf
from torch.utils.data import Dataset, DataLoader
from typing import Any, Callable, Dict, IO, List, Optional, Tuple, Union
import lightnet as ln
import re
from skimage import io, transform

__all__ = ['VoltaDaUfes']
log = logging.getLogger('lightnet.VoltaDaUfes.dataset')

class VoltaDaUfes(Dataset):
    """ VoltaDaUfes Dataset """

    def __init__(self, image_list, params, augment):
        
        # Data transformation pipeline
        if augment:
            transform = ln.data.transform.Compose([
                lambda img: img.convert('RGB'),
                ln.data.transform.RandomHSV(params.hue, params.saturation, params.value),
                ln.data.transform.RandomJitter(params.jitter, fill_color=0),
                ln.data.transform.RandomRotate(params.angle),
                ln.data.transform.Letterbox(dataset=self, fill_color=0),
                ln.data.transform.FitAnno(),
                tf.ToTensor(),
            ])
        else:
            transform = ln.data.transform.Compose([
                lambda img: img,
                tf.ToTensor(),
            ])
        with open(image_list) as file:
            self.image_list = [line.strip() for line in file]
        self.county=len(self.image_list)        
        self.transform = transform
        self.params = params

    def __len__(self) -> int:
        return self.county
    
    def __getitem__(self, idx) -> Tuple[Any, Any]:
        img_name = self.image_list[idx]
        image = io.imread(img_name)
        class_id = int(img_name.split("_")[-1].split(".")[-2].replace('B','').replace('E',''))
        if self.transform:
            sample = self.transform(image)

        return sample,class_id        


# **4 - Definir a classe principal**

In [9]:
#!/usr/bin/env python
import os
import argparse
import logging
from statistics import mean
import torch
from PIL import Image
import pandas as pd
from tqdm import tqdm
import lightnet as ln
import brambox as bb
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

torch.set_num_threads(8)
log = logging.getLogger('lightnet.VoltaDaUfes.test')


# TODO : refactor this to a simple set of functions, as it is overkill to create this class.
class DarknetEngine:
    def __init__(self, params, test_loader,train_loader, **kwargs):
        self.params = params
        self.test_loader = test_loader
        self.train_loader = train_loader

        self.network = params.network
        self.train_loss = []
        # Setting kwargs
        for k, v in kwargs.items():
            if not hasattr(self, k):
                setattr(self, k, v)
            else:
                log.error('{k} attribute already exists on TestEngine, not overwriting with `{v}`')

    def __call__(self):
        self.params.to(self.device)
        self.network.eval()

    def train(self,epochs,log_interval):
        params.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(params.optimizer, len(self.train_loader))
        val = params.steps
        for epoch in range(epochs):
            if epoch>0 and epoch % val == 0:
                val = val + val*params.multiply_sgdr
                total = val * len(self.train_loader)
                params.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(params.optimizer, total)    
            self.train_epoch(log_interval,epoch,val)
            self.test()
            if epoch>0 and epoch % 1 == 0:
                self.params.save(os.path.join('backup', f'deepvgl_{epoch}.state.pt'))
                self.params.network.save(os.path.join('backup', f'deepvgl_{epoch}.state.weights'))

    def train_epoch(self,log_interval,epoch,step):
        params.network.train()
        local_loss = 0
        for batch_idx,(data, target) in enumerate((self.train_loader)):
            data, target = data.to(self.device), target.to(self.device)
            params.optimizer.step()
            params.optimizer.zero_grad()
            output = self.network(data)
            loss_temp = self.params.loss(output, target)
            loss_temp.backward()
            self.train_loss.append(loss_temp.item())
            if ((batch_idx+1) * params.mini_batch_size) % params.batch_size == 0:
                local_loss = mean(self.train_loss[-args.subdivisions:])
                self.train_loss = []
            real_batch_id = ((batch_idx+1) * params.mini_batch_size) / params.batch_size
            if real_batch_id>0 and real_batch_id % log_interval == 0:
                print('Train Epoch: {} [{:5d}/{} ({:.0f}%)]\tLoss: {:.6f}  \tLearning Rate: {:.6f}'.format(
                    epoch, (batch_idx+1) * len(data), len(self.train_loader.dataset),
                    100. * batch_idx / len(self.train_loader), 
                    local_loss,
                    params.scheduler.get_last_lr()[0]))
            params.scheduler.step()


    def test(self):
        pred, label = [],[]
        images, labels = next(iter(self.test_loader))
        counter = 0
        with torch.no_grad():
            for data, target in self.test_loader:
                data = data.to(self.device)
                output = self.network(data)
                probs=nn.Softmax(dim=1)
                softmax_out = probs(output)
                counter+=1
                print(counter)
                dvgl = (softmax_out.argmax(dim=1))
                pred.append(dvgl)
                label.append(target)

        print("deep_vgl\tGT\tMAE0\tMAE1\tMAE2")
        MAE = np.zeros(3)
        for (dvgl,gt) in zip(pred,label):
            
            if abs(dvgl[0].item() - gt[0].item()) == 0:
                MAE[0]=MAE[0]+1
                MAE[1]=MAE[1]+1
                MAE[2]=MAE[2]+1
            if abs(dvgl[0].item() - gt[0].item()) == 1:
                MAE[1]=MAE[1]+1
                MAE[2]=MAE[2]+1
            if abs(dvgl[0].item() - gt[0].item()) == 2:
                MAE[2]=MAE[2]+1
            
            print(dvgl[0].item(),"\t\t",gt[0].item(),"\t", MAE[0], MAE[1], MAE[2] )

        MAE = 100*MAE/(self.test_loader.__len__())
        print("MAE0\tMAE1\tMAE2")
        print("%2.1f" % MAE[0], "\t%2.2f" % MAE[1], "\t%2.2f" % MAE[2] )


parser = argparse.ArgumentParser(
    description='Test trained network',
    formatter_class=argparse.ArgumentDefaultsHelpFormatter
)
# parser.add_argument('initial_weight', help='Path to initial weights file')
# parser.add_argument('-n', '--network', help='network config file', required=True)
# parser.add_argument('-c', '--cuda', help='Use cuda', action='store_true')
# parser.add_argument('-l', '--labels', help='labels file', required=True)
parser.add_argument('-b','--batch_size', type=int, default=128, help='input batch size for training (default: 64)')
parser.add_argument('-d','--subdivisions', type=int, default=16, help='input batch subdivisions (default: 16)')
parser.add_argument('-e','--epochs', type=int, default=130, help='number of epochs to train (default: 130)')
parser.add_argument('-r','--learning_rate', type=float, default=0.0001, help='learning rate (default: 0.001)')
parser.add_argument('-j','--jitter', type=float, default=0.3, help='jitter (default: 0.3)')
parser.add_argument('-a','--angle', type=float, default=7, help='random angle (default: 7)')
parser.add_argument('-v','--value', type=float, default=1.5, help='HSV value (default: 1.5)')
args = parser.parse_args(args=[])


weights = '/dados/darknet/darknet19_448.conv.23'
labels = '/dados/ufes/labels_3_1.txt'
# Parse arguments
device = torch.device('cpu')
validation=True

if torch.cuda.is_available():
    log.debug('CUDA enabled')
    device = torch.device('cuda')
else:
    log.error('CUDA not available')


#params = ln.engine.HyperParameters.from_file(args.network)
new_network(load_classes(labels))
params = params
params.class_label_map = load_classes(labels)
params.batch_size = args.batch_size
params.mini_batch_size=int(args.batch_size/args.subdivisions)
print(repr(params.network))
if weights is not None:
    if weights.endswith('.state.pt'):
        params.load(weights)
    else:
        params.network.load(weights, strict=False)
#params.network.load(args.initial_weight)



# Optimizer
params.optimizer = torch.optim.SGD(
    params.network.parameters(),
    lr = args.learning_rate,
    momentum = 0.9,
    weight_decay = .0005
)
params.multiply_sgdr = 2
params.steps = 1
params.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(params.optimizer, params.steps)

# Dataloader
data = params.train_set
train_dataloader = torch.utils.data.DataLoader(
    VoltaDaUfes(data, params, False),
    batch_size = params.mini_batch_size,
    pin_memory=True,
    shuffle = True
    )

# Dataloader
data = params.val_set
testing_dataloader = torch.utils.data.DataLoader(
    VoltaDaUfes(data, params, False),
    batch_size = 1,
    shuffle = False
    )

# Start test  (params, test_loader,train_loader,)
eng = DarknetEngine(
    params, testing_dataloader,train_dataloader,
    device=device
)
eng()


usage: ipykernel_launcher.py [-h] [-b BATCH_SIZE] [-d SUBDIVISIONS]
                             [-e EPOCHS] [-r LEARNING_RATE] [-j JITTER]
                             [-a ANGLE] [-v VALUE]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/thiago/.local/share/jupyter/runtime/kernel-e26c83d7-c3ce-4833-9808-d2e40b585858.json


SystemExit: 2

# **5 - Executar Treino**

In [ ]:
# train( epocas, log_interval )
eng.train(5,1)

# **6 - Executar Teste**
resultado esperado
MAE0	MAE1	MAE2
76.7 	96.11 	97.75

In [ ]:
# weights = "/dados/ufes/deepvgl_final.weights"
weights = "backup/deepvgl_5.state.pt"
if weights is not None:
    if args.initial_weight.endswith('.state.pt'):
        params.load(weights)
    else:
        params.network.load(weights, strict=False)

eng.test()